**Setup:**

In [8]:
%pip install -Uqq fastai torchvision

from fastai.vision.all import *

Note: you may need to restart the kernel to use updated packages.


Importing Data:

In [9]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Look at `train_data`:

In [10]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

Show shape of `train_data`:

In [11]:
train_data.data.shape

torch.Size([60000, 28, 28])

Create Dataloaders `dl` and `valid_dl` with batch size 100:

In [12]:
dl = DataLoader(train_data, batch_size=100, shuffle=True)
valid_dl = DataLoader(test_data, batch_size=100, shuffle=True)

Import `Net` from `learning_functions.py`:

In [13]:
from learning_functions import Net
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available
neural_net = Net().to(device)

Define functions `train` and `test` and define a loss function and an optimizer with learning rate 0.2:

In [14]:
from torch import nn

optimizer = SGD(neural_net.parameters(), lr=0.2)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    neural_net.train()
    global optimizer
    for batch_idx, (data, target) in enumerate(dl):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = neural_net(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(dl.dataset)} ({100. * batch_idx / len(dl):.0f}%)]\t{loss.item():.6f}")

def test():
    neural_net.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in valid_dl:
            data, target = data.to(device), target.to(device)
            output = neural_net(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(valid_dl.dataset)
    print(f'\nTest Loss: Average Loss: {test_loss:.4f}, Accuracy: {correct}/{len(valid_dl.dataset)} {100. * correct / len(valid_dl.dataset):.0f}%')

Train Model for 100 epochs:

In [16]:
for epoch in range(1, 101):
        train(epoch)
        test()

Train Epoch: 1 [0/60000 (0%)]	1.605512
Train Epoch: 1 [2000/60000 (3%)]	1.693357
Train Epoch: 1 [4000/60000 (7%)]	1.634118
Train Epoch: 1 [6000/60000 (10%)]	1.625600
Train Epoch: 1 [8000/60000 (13%)]	1.623009
Train Epoch: 1 [10000/60000 (17%)]	1.592018
Train Epoch: 1 [12000/60000 (20%)]	1.681488
Train Epoch: 1 [14000/60000 (23%)]	1.615613
Train Epoch: 1 [16000/60000 (27%)]	1.597578
Train Epoch: 1 [18000/60000 (30%)]	1.567154
Train Epoch: 1 [20000/60000 (33%)]	1.617719
Train Epoch: 1 [22000/60000 (37%)]	1.612946
Train Epoch: 1 [24000/60000 (40%)]	1.565437
Train Epoch: 1 [26000/60000 (43%)]	1.618703
Train Epoch: 1 [28000/60000 (47%)]	1.586755
Train Epoch: 1 [30000/60000 (50%)]	1.665592
Train Epoch: 1 [32000/60000 (53%)]	1.588227
Train Epoch: 1 [34000/60000 (57%)]	1.579021
Train Epoch: 1 [36000/60000 (60%)]	1.586362
Train Epoch: 1 [38000/60000 (63%)]	1.614133
Train Epoch: 1 [40000/60000 (67%)]	1.599554
Train Epoch: 1 [42000/60000 (70%)]	1.568233
Train Epoch: 1 [44000/60000 (73%)]	1.537986

Export Predicition Class:

In [20]:
torch.save(neural_net.state_dict(), 'mnist_identifier.pth')